### 10년치 편의점 판매 데이터 분석하기
- 1500 만건
- 특정 상품을 정해 시간당 몇 개씩 팔리는 지 분석
- 확률실험 관측값(팔린갯수)를 확률변수로 사용

In [1]:
# 대량의 데이터 빠르게 
library(data.table)

In [2]:
df <- fread('r-ggagi-data/example_conveniencestore.csv',
            encoding='UTF-8',
           data.table=F)

In [3]:
head(df)

Date,sellproduct,company,payment,margin
2006-01-01 07:00:02,보가리스웨트 620미리리터(P),동화오츠카㈜,체크카드,0.11
2006-01-01 07:00:14,막스 500미리리터(캔),블랙맥주㈜,현금,0.04
2006-01-01 07:00:42,맛좋은우유 GT 1리터(팩),서양유업㈜,체크카드,0.13
2006-01-01 07:00:48,육개장사발탕면 86g(컵),㈜낭심,신용카드,0.28
2006-01-01 07:01:14,제주도삼다수<U+00A0>2리터(P),㈜낭심,신용카드,0.22
2006-01-01 07:01:24,경기도장수생막걸리 750미리리터(P),경기도탁주제조협회,현금,0.30


In [4]:
# 표본 만들기 - sample()
S <- df[sample(nrow(df), 5000), ]

In [5]:
str(S)

'data.frame':	5000 obs. of  5 variables:
 $ Date       : chr  "2013-06-14 19:46:59" "2012-12-10 07:55:39" "2015-05-10 04:30:12" "2011-10-16 15:37:30" ...
 $ sellproduct: chr  "굿스맥주 355미리리터(캔)" "막스 500미리리터(캔)" "가스라이트큐팩<U+00A0>1.6리터(P)" "굿스맥주 355미리리터(캔)" ...
 $ company    : chr  "오비이락맥주㈜" "블랙맥주㈜" "오비이락맥주㈜" "오비이락맥주㈜" ...
 $ payment    : chr  "교통카드" "신용카드" "신용카드" "체크카드" ...
 $ margin     : num  0.21 0.04 0.17 0.21 0.26 0.27 0.3 0.14 0.21 0.32 ...


In [6]:
#  분석 상품 선택 - 가장 많이 팔린 상품
sort(table(S$sellproduct))


          보가리스웨트 620미리리터(P)          바리스타라떼 250미리리터(컵) 
                                   33                                    37 
     아이리시스<U+00A0>500미리리터(P) 글라소비타민물공급<U+00A0>500미리리터(P) 
                                   39                                    42 
           딸기짱우유 240미리리터(팩)      조르지아오리지널 240미리리터(캔) 
                                   42                                    44 
    TOP스위트블랙커피 275미리리터(캔)            아이리시스<U+00A0>2리터(P) 
                                   45                                    47 
     가스라이트큐팩<U+00A0>1.6리터(P)                    세계콘 170미리리터 
                                   49                                    49 
         굿이브닝케어 100미리리터(병)                육개장사발탕면 86g(컵) 
                                   50                                    50 
     팔성사이다<U+00A0>500미리리터(P)            경기도우유 200미리리터(팩) 
                                   50                                    51 
              맛좋은우유 GT 1리터(팩)               메지밀B 190미리리터(

In [7]:
library(stringr)

In [8]:
head(S$Date,3)

[1] "2013-06-14 19:46:59" "2012-12-10 07:55:39" "2015-05-10 04:30:12"

In [9]:
S$time <- str_split_fixed(S$Date, " ", 2)[,2]
S$hour <- str_split_fixed(S$time, ":", 3)[,1]
head(S, 1)

,Date,sellproduct,company,payment,margin,time,hour
11755751,2013-06-14 19:46:59,굿스맥주 355미리리터(캔),오비이락맥주㈜,교통카드,0.21,19:46:59,19


### 묶음행사 상품 판매 데이터 분석하기
- 이벤트 상품 8개 중 6개를 골라 세트 구입
- 중복해서 구입 가능 => 중복 조합
- 6개 세트에서 마진이 좋은 상품도, 나쁜 상품도 있음
- 마진의 기준 0.13
- 6개 상품의 마진이 0.13인 상품갯수의 확률변수 X = {0,1,2,3,4,5,6}
- 이때의 확률변수의 분포